<a href="https://colab.research.google.com/github/Yewon9/STT/blob/main/Wav2Vec2_Training_General_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets soundfile

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Trainer
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
from transformers import TrainingArguments, Trainer

# DATA

In [4]:
general = pd.read_csv('general_eda.csv')
general

,Path,Text,Audio_Length,Cleaned_Text,Text_Length
0,/content/drive/MyDrive/project/data_aihub/4/5e...,편하게 좋은 곳으로 갔겠지? 응. 힘을 내야되는데 어떡하지.,9.259,편하게 좋은 곳으로 갔겠지? 응. 힘을 내야되는데 어떡하지.,33
1,/content/drive/MyDrive/project/data_aihub/5_2/...,내가 받은 선물이 궁금한거야? 보고싶어? 그래. 나중에 보여줄게.,8.661,내가 받은 선물이 궁금한거야? 보고싶어? 그래. 나중에 보여줄게.,36
2,/content/drive/MyDrive/project/data_aihub/4/5e...,아직 말씀 안 드렸어. 어떡하지.,5.547,아직 말씀 안 드렸어. 어떡하지.,18
3,/content/drive/MyDrive/project/data_aihub/5/5e...,오늘도 털어놓았더니 마음이 한결 가벼워졌어. 다음에도 또 부탁해.,8.277,오늘도 털어놓았더니 마음이 한결 가벼워졌어. 다음에도 또 부탁해.,36
4,/content/drive/MyDrive/project/data_aihub/5_2/...,그건 나도 몰라.,2.347,그건 나도 몰라.,9
...,...,...,...,...,...
11995,/content/drive/MyDrive/project/data_aihub/4/5e...,3년 전에 동호회 갔었는데 거기서 처음 만났지.,6.955,3년 전에 동호회 갔었는데 거기서 처음 만났지.,26
11996,/content/drive/MyDrive/project/data_aihub/4/5e...,깜짝 놀랬다니까.,4.053,깜짝 놀랬다니까.,9
11997,/content/drive/MyDrive/project/data_aihub/5_2/...,실망스러운건 사과를 억지로 했다는 거야. 상처받았지.,5.419,실망스러운건 사과를 억지로 했다는 거야. 상처받았지.,29
11998,/content/drive/MyDrive/project/data_aihub/5/5e...,합격자 명단에 이름이 또 없는 거 있지?,4.523,합격자 명단에 이름이 또 없는 거 있지?,22


In [5]:
dataset = Dataset.from_pandas(general)
print(dataset)

Dataset({
    features: ['Path', 'Text', 'Audio_Length', 'Cleaned_Text', 'Text_Length'],
    num_rows: 12000
})


# Wav2Vec2

In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [7]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["Path"])
    batch["speech"] = speech_array
    return batch

In [ ]:
dataset = dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [ ]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate = 16000).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["Cleaned_Text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns = ["speech"])

In [ ]:
dataset = dataset.train_test_split(test_size = 0.1)

In [ ]:
training_args = TrainingArguments(
    output_dir = "./wav2vec2",
    evaluation_strategy = "epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    logging_steps = 10,
    learning_rate = 1e-4,
    num_train_epochs = 3,
    save_steps = 500,
    save_total_limit = 2,
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    tokenizer = processor.feature_extractor,
)

In [ ]:
trainer.train()